In [ ]:
from torch import FloatTensor, LongTensor, Tensor
import math

## Module Superclass

In [ ]:
class Module(object):
    def forward(self, *input_):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []

## ReLU Module

ReLU function: 
\begin{equation}
f(x) = max(0, x)
\end{equation}

the derivative of ReLU is

\begin{equation} 
f'(x)=
    \begin{cases}
      1, & \text{if}\ x>0 \\
      0, & \text{otherwise}
    \end{cases}
\end{equation}

In [ ]:
# This module represents the ReLU activation function
class ReLU(Module):
    def __init__(self):
        self.z = None
    
    # input_: the tensor outputed by the current layer
    def forward(self, input_):
        self.z = input_.clone()
        input_[input_ < 0] = 0
        return input_
        
    def backward(self, gradwrtoutput):
        da = gradwrtoutput
        tensor = self.z.clone()
        # g'(z)
        tensor[tensor > 0] = 1
        tensor[tensor < 0] = 0
        # dz[l]
        return da.mul(tensor)
        
    def param(self):
        return []
    
    def zero_grad(self):
        pass

## Tanh Module

In [ ]:
class Tanh(Module):   
    def __init__(self):
        self.z = None
    
    # input_: the tensor outputed by the current layer
    def forward(self, input_):
        self.z = input_
        return input_.tanh()
        
    def backward(self, gradwrtoutput):
        da = gradwrtoutput
        # g'(z)
        g_prime = (1 - self.z.tanh().pow(2))
        # dz[l]
        return da.mul(g_prime)
        
    def param(self):
        return []
    
    def zero_grad(self):
        pass

## Linear Module
fully connected layer

In [ ]:
class Linear(Module):   
    def __init__(self, in_dim, out_dim):
        # keep track of the weigths, the biases and the output of the previous layer's activation function
        self.w = Tensor(out_dim,in_dim).normal_(0)
        self.b = Tensor(out_dim,1).normal_(0)
        self.x_previous_layer = None
        # init the gradient of the loss wrt w / b
        self.grad_w_sum = Tensor(self.w.size()).zero_()
        self.grad_b_sum = Tensor(self.b.size()).zero_()
    
    # input_: the output of the previous layer's activation function
    def forward(self, input_):
        self.x_previous_layer = input_
        return (self.w.mm(input_.t()) + self.b).t()
        
    def backward(self, gradwrtoutput):
        dz = gradwrtoutput.t()
        dw = dz.mm(self.x_previous_layer)
        db = dz
        # sum the gradients for the weights and biases
        self.grad_w_sum += dw
        self.grad_b_sum += db.sum(1).unsqueeze(1)
        return (self.w.t().mm(dz)).t()
        
    # returns a list of pairs, each composed of a parameter tensor and a gradient tensor
    # parameters: weights and biases
    def param(self):
        return [ (self.w, self.grad_w_sum), (self.b, self.grad_b_sum) ]
    
    def zero_grad(self):
        self.grad_w_sum.zero_()
        self.grad_b_sum.zero_()

## Sequential Module
to combine several modules in basic sequential structure

In [ ]:
# This module allows to combine several modules (layers, activation functions) in a basic sequential structure
class Sequential(Module):    
    def __init__(self, *layers_):
        self.modules = layers_
        
    # input_: the input data is a minibatch whose columns are features and lines are samples
    def forward(self, input_):
        x = input_
        for module in self.modules:
            x = module.forward(x)
        return x
        
    def backward(self, gradwrtoutput):
        x = gradwrtoutput
        for module in reversed(self.modules):
            x = module.backward(x)
        return x
        
    # returns a flatened list of each module's parameters
    # each parameter in the list is represented as a tuple containing the parameter tensor (e.g. w)
    # and the gradient tensor (e.g. dl/dw)
    def param(self):
        return [ p for module in self.modules for p in module.param() ]
    
    # s,
    def zero_grad(self):
        for module in self.modules:
            module.zero_grad()

## MSE Loss Function

In [ ]:
class LossMSE(Module): 
    def __init__(self):
        self.error = None
        
    def forward(self, preds, labels):
        self.error = preds - labels
        return self.error.pow(2).sum()
        
    def backward(self):
        return 2 * self.error
        
    def param(self):
        return []

## SGD optimization

In [ ]:
class optim_SGD(Module):
    # parameters: the parameters of the Sequential module
    def __init__(self, parameters, learning_rate):
        self.param = parameters #[ p.shallow() for tup in parameters for p in tup ]
        self.lr = learning_rate
        
    # performs a gradient step (SGD) for all parameters
    def step(self):
        for (p, grad_p) in self.param:
            p.sub_(self.lr*grad_p)

## Helpers

In [ ]:
def generate_disc_set(nb):
    a = Tensor(nb, 2).uniform_(0, 1)
    target = ((a.pow(2).sum(1)).sqrt() < math.sqrt(1/(2*math.pi))).long()
    return a, target

In [ ]:
# converts 'target' Tensor to one hot labels
def convert_to_one_hot(target):
    tmp = FloatTensor(target.size(0), 2).fill_(0)
    for k in range(0, target.size(0)):
        tmp[k, train_target[k]] = 1
    return tmp

In [ ]:
def compute_accuracy(model, input_, target):
    nb_data_errors = 0
    output = model.forward(input_)

    _, predicted_classes = output.max(1)
    for k in range(input_.size(0)):
        if target[k] != predicted_classes[k]:
            nb_data_errors = nb_data_errors + 1
    return 100 - (100*(nb_data_errors / input_.size(0)))

In [37]:
def train_model(train_input, train_one_hot_target, model, criterion, optimizer, nb_epochs=100, mini_batch_size=5, verbose=False):
    for e in range(0, nb_epochs):
        loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model.forward(train_input.narrow(0, b, mini_batch_size))
            # sum the loss for each batch to get the current epoch's loss
            loss += criterion.forward(output, train_one_hot_target.narrow(0, b, mini_batch_size))
            # set the gradients of all layers to zero before the next batch can go through the network
            model.zero_grad()
            model.backward(criterion.backward())
            optimizer.step() # performs a gradient step to optimize the parameters
        if verbose:
            print("Epoch", e+1, ":", loss)

## Test file

In [38]:
# generate training and testing data
train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

# convert targets to one hot labels
train_one_hot_target = convert_to_one_hot(train_target)
test_one_hot_target = convert_to_one_hot(test_target)

In [40]:
nb_epochs = 100
mini_batch_size = 5
model = Sequential(Linear(2,25), Tanh(), Linear(25,25), Tanh(), Linear(25, 25), Tanh(), Linear(25,2))
criterion = LossMSE()
optimizer = optim_SGD(model.param(), 1e-3)

train_model(train_input, train_one_hot_target, model, criterion, optimizer, nb_epochs, mini_batch_size, verbose=True)

print("Train accuracy :", compute_accuracy(model, train_input, train_target), "%")
print("Test accuracy :", compute_accuracy(model, test_input, test_target), "%")

Epoch 1 : 832.9720829972371
Epoch 2 : 154.90995721241262
Epoch 3 : 129.86513044108133
Epoch 4 : 110.14496682553451
Epoch 5 : 96.83185583888873
Epoch 6 : 88.59321982804663
Epoch 7 : 82.43121548430342
Epoch 8 : 77.32139528462334
Epoch 9 : 72.71635719806248
Epoch 10 : 68.59061387278207
Epoch 11 : 64.51636541251641
Epoch 12 : 60.313300845997006
Epoch 13 : 56.39257784141226
Epoch 14 : 53.68016271212343
Epoch 15 : 50.6784058541575
Epoch 16 : 48.16249963307973
Epoch 17 : 46.09611943953415
Epoch 18 : 44.31350497681193
Epoch 19 : 42.750062040166156
Epoch 20 : 41.37042179608008
Epoch 21 : 40.14095954670577
Epoch 22 : 39.036341381068965
Epoch 23 : 38.04109333754259
Epoch 24 : 37.1443383343493
Epoch 25 : 36.335787241992634
Epoch 26 : 35.60435188836871
Epoch 27 : 34.93895581631742
Epoch 28 : 34.329268978727555
Epoch 29 : 33.766434263849646
Epoch 30 : 33.243015057384156
Epoch 31 : 32.75315915758428
Epoch 32 : 32.29219931610962
Epoch 33 : 31.856454960266916
Epoch 34 : 31.442975512965376
Epoch 35 : 31

In [41]:
n_iters = 50
test_acc=[]
train_acc=[]
for i in range(n_iters):
    print('iter', i+1)
    model = Sequential(Linear(2,25), Tanh(), Linear(25,25), Tanh(), Linear(25, 25), Tanh(), Linear(25,2))
    criterion = LossMSE()
    optimizer = optim_SGD(model.param(), 1e-3)
    
    train_model(train_input, train_one_hot_target, model, criterion, optimizer, nb_epochs, mini_batch_size)

    train_a = compute_accuracy(model, train_input, train_target)
    test_a = compute_accuracy(model, test_input, test_target)
    print("Train accuracy :", train_a, "%")
    print("Test accuracy :", test_a, "%")
    train_acc.append(train_a)
    test_acc.append(test_a)

iter 1


TypeError: sub received an invalid combination of arguments - got (torch.LongTensor), but expected one of:
 * (float value)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.LongTensor[0m)
 * (torch.FloatTensor other)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.LongTensor[0m)
 * (float value, torch.FloatTensor other)


In [ ]:
print('train accuracy', sum(train_acc)/len(train_acc))
print('test accuracy', sum(test_acc)/len(test_acc))